In [1]:
import io
import re
import numpy as np
import pandas as pd
import numpy.linalg as lin
import warnings 
from gensim.models import Word2Vec
import pickle
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
warnings.filterwarnings('ignore')

In [2]:
stopwords=['ଦେଇଛନ୍ତି', 'ଲେଖାଏଁ', 'ଜଣେ', 'ଏହା', 'ତେଣୁ', 'ମିଳିଥାଏ', 'ପାଇଁ', 'ନେଉଛନ୍ତି', 'ଯୋଗୁଁ', 'ଏପର୍ଯ୍ୟନ୍ତ', 'ଏଭଳି', 'କରୁଛି', 'ଓ', 'ଯାଏଁ','ହୋଇଛନ୍ତି', 'କି', 'କରାଯିବା', 'ପରେ', 'ଏହି', 'ଏବଂ', 'ଜଣ', 'ଥିବା','ହୋଇଥିଲା', 'ତେବେ', 'ଆଜି', 'ଜଣଙ୍କ', 'ଏଥି', 'ଗତ', 'ହୋଇଥିଲେ',
       'ଦିନ ଧରି', 'ହେଉଥିବା', 'ଦିନ ତଳେ', 'ଯୋଗେ', 'ବୋଲି', 'ଜଣାପଡ଼ିଛି',
       'ଦ୍ବାରା', 'କରି', 'ଯାଇ', 'ଏ ନେଇ', 'ଚାଲୁ', 'ରହିଛି', 'ତାରିଖ',
       'ମିଳିଛି', 'ବର୍ଷୀୟ', 'ଦିନ ତଳେ', 'ସହ', 'ଆସିଛନ୍ତି', 'ମଧ୍ୟ', 'କେଉଁ',
       'ହୋଇଯାଇଛି', 'ନେଇ ଯାଇଛି', 'କାମ', 'କରିଛି', 'ହେବା', 'ପରେ', 'ଏବେ',
       'ହୋଇଛି', 'ତେବେ', 'ରଖି', 'ନିଆଯାଇଥିବା', 'ପର୍ଯ୍ୟନ୍ତ', 'କରିଛନ୍ତି',
       'ଉପରେ', 'ଦେଲେ', 'ଥର', 'ଏବଂ', 'ଆଉ']
alphabets=['ଅ','ଆ','ଇ','ଈ','ଉ','ଊ','ଋ','ଌ','ଏ','ଐ','ଓ','ଔ','କ','ଖ','ଗ','ଘ','ଙ','ଚ','ଛ','ଜ','ଝ','ଞ','ଟ','ଠ','ଡ','ଢ','ଣ',
 'ତ','ଥ','ଦ','ଧ','ନ','ପ','ଫ','ବ','ଭ','ମ','ଯ','ର','ଲ','ଳ','ଵ','ଶ','ଷ','ସ','ହ','ଡ଼','ଢ଼','ୟ','ୠ','ୡ','ୱ']
matra=['ଁ','ଂ','ଃ','଼','ା','ି','ୀ','ୁ','ୂ','ୃ','ୄ','େ','ୈ','ୋ','ୌ','୍','ୢ','ୣ']

In [3]:
delimiters=["।","?","!"]

In [4]:
gloveFile=open("glove/or-d100-glove.txt","r",encoding='utf-8')
glove={}
c=1000
for i in gloveFile:
    i=i.split(" ")
    glove[i[0]]=np.array(list(map(float,i[1:])))

In [5]:
def o2h(text):
    return transliterate(text, sanscript.ORIYA, sanscript.DEVANAGARI)

In [6]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra+delimiters:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())
def sent_tokenize(text):
    text=remove_special_characters(text)
    regexPattern = '|'.join(map(re.escape, delimiters))
    sentences=re.split(regexPattern,text)
    return [sent.strip() for sent in sentences if len(sent.strip())>0]

In [7]:
def word_embedding(word):
    try:
        return glove[o2h(word)]
    except:
        return np.array([0.0]*100)

In [8]:
def sentence_embedding(sentence):
    sentence=remove_special_characters(sentence)
    words=sentence.split()
    words=[word  for word in words if word not in stopwords]
    if len(words)>0:
        sentence_embedding=[word_embedding(word) for word in words]
        return np.array(list(map(lambda x: sum(x)/len(x), zip(*sentence_embedding))))
    return np.array([0.0]*100)

In [9]:
data=pd.read_csv('../odia.csv')

# Sentence By Sentence

In [10]:
verse_embeddings_sentence=[]
verse_embeddings_max=[]
verse_embeddings_mean=[]
for i in range(len(data)):
    text=data.loc[i]['Commentary']
    sentences=sent_tokenize(text)
    embeddings=[sentence_embedding(sentence) for sentence in sentences]
    #sentence
    verse_embeddings_sentence.append(embeddings)
    #Max Pooling
    norms=[lin.norm(i) for i in embeddings]
    index=norms.index(max(norms))
    verse_embeddings_max.append(embeddings[index])
    #Mean Pooling
    embeddings=np.array(list(map(lambda x: sum(x)/len(x), zip(*embeddings))))
    verse_embeddings_mean.append(embeddings)

# Whole

In [11]:
verse_embeddings_whole=[]
for i in range(len(data)):
    text=data.loc[i]['Commentary']
    embeddings=sentence_embedding(text)
    verse_embeddings_whole.append(embeddings)

In [12]:
file=open('sentence.pkl','wb')
pickle.dump(verse_embeddings_sentence,file)
file.close()
file=open('max.pkl','wb')
pickle.dump(verse_embeddings_max,file)
file.close()
file=open('mean.pkl','wb')
pickle.dump(verse_embeddings_mean,file)
file.close()

In [13]:
file=open('whole.pkl','wb')
pickle.dump(verse_embeddings_whole,file)
file.close()

In [14]:
file=open('glove.pkl','wb')
pickle.dump(glove,file)
file.close()